In [ ]:
!pip -q install rarfile

In [ ]:
import zipfile, os
zip_ref=zipfile.ZipFile('fixed-15.zip','r')
zip_ref.extractall('fixed')


In [ ]:
# مسیر ها رو برای ساینز بگه
import os

def find_signs_files(root_folder):
    signs_files = []

    for root, dirs, files in os.walk(root_folder):
        if "signs.txt" in files:
            signs_file_path = os.path.join(root, "signs.txt")
            signs_files.append(signs_file_path)

    return signs_files

root_folder_path = "fixed data"

signs_files_list = find_signs_files(root_folder_path)

for signs_file_path in signs_files_list:
    print(signs_file_path)

In [ ]:
# مرتب سازی سطر و ستون نزولی

import os
import pandas as pd

def find_and_parse_signs_files(root_folder):
    results = {}

    for root, dirs, files in os.walk(root_folder):
        for file in files:
            if file == "signs.txt":
                file_path = os.path.join(root, file)
                parsed_data = parse_signs_file(file_path)

                disease_name = os.path.basename(os.path.dirname(os.path.dirname(file_path)))

                results[disease_name] = parsed_data

    return results

def parse_signs_file(file_path):
    result_list = []

    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

            for line in lines:
                if '.' in line and ':' in line and line.index('.') < line.index(':'):
                    first_split = line.split('.')[1]
                    final_result = first_split.split(':')[0].strip()
                    final_result = final_result.replace('\u200c', '')
                    result_list.append(final_result)

    except FileNotFoundError:
        print(f"خطا: فایل {file_path} یافت نشد.")
    except Exception as e:
        print(f"خطا: {e}")

    return result_list

root_folder_path = "fixed data"

result_dict = find_and_parse_signs_files(root_folder_path)

df = pd.DataFrame()

for disease_name, signs_list in result_dict.items():
    for sign in signs_list:
        df.loc[disease_name, sign] = 1

df = df.fillna(0)

# df.loc["سالم", :] = 0

df.to_csv('signs_t.csv')


In [3]:
# مرتب سازی سطر و ستون

import pandas as pd

df = pd.read_csv('signs_t.csv', index_col=0)

df_user = pd.DataFrame(index=df.index)

n = int(input('enter how many quest: ')) # how many quest
for i in range(n):

    df = df.reindex(df.sum().sort_values(ascending=False).index, axis=1)

    df = df.reindex(df.sum(axis=1).sort_values(ascending=False).index)

    sign = df.columns[0]

    answer = input(f'آیا شما علامت {sign} را دارید؟ (1: بله, 0: خیر)')

    df_user[sign] = df[sign]
    df_user.loc['user', sign] = answer

    df = df.drop(columns=[sign])

df_user.to_csv('signs_u.csv')

In [4]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('signs_u.csv')

data.set_index(data.columns[0], inplace=True)
data = data.astype(float)

csim = cosine_similarity(data)

sim_df = pd.DataFrame(csim, index=data.index, columns=data.index)


sorted_similarities = sim_df['user'].sort_values(ascending=False)

sorted_similarities = (sorted_similarities * 100).drop('user')

for index, value in sorted_similarities.items():
    print(f'{value:.2f}%: {index}')

max_similarity = sorted_similarities.max()
if max_similarity < 40:
    print('کاربر سالم است.')
elif 40 <= max_similarity < 70:
    print(f'کاربر مریض است. بیماری ممکن: {sorted_similarities.idxmax()}')
else:
    print(f'کاربر باید به پزشک مراجعه کند. بیماری ممکن: {sorted_similarities.idxmax()}')

100.00%: sang koliye
100.00%: reflax mede
70.71%: sardi mede
70.71%: sharyanhaye kroneri
57.74%: migren sard
57.74%: high triglycerides
44.72%: aritmy ghalb
0.00%: high pressure
0.00%: exema
0.00%: high cholesterol
0.00%: lupus
0.00%: naresayi ghalb
0.00%: saratan poost
0.00%: sardi koliye
0.00%: migren garm
کاربر باید به پزشک مراجعه کند. بیماری ممکن: sang koliye
